# workflow

is it the same data as https://github.com/anastassiavybornova/knudepunkter/blob/main/src/wfs_func.py#L87 ?

currently cf simplification_api_knupu notebook in simplification

* install sgeop bleeding edge
* import all raw edges
* to proj crs
* buffer around all edges (custom, now: 20m)
* union of buffer polygons
* get boundary and explode it
* drop too-short linestrings (loops within)
* polygonize
* only keep those polygons that have interiors
* now we have one polygon per network compoentn
* for each polygon:
	* get delineation (all interiors and the exterior)
	* unzip delineation lines, with modulo caveat not to lose geoms
	* skeletonize (delineation lines and comp poly) with sgeop.geometry.voronoi_skeleton
	* convert to nx
	* should be only 1 component!! (#TODO : consolidate nodes with sgeop.nodes.consolidate_nodes OR shapely.snap?) 
	* iteratively remove degree-one nodes if dangling edge is short enough
	* remove false nodes
	* convert back to nx
* combine into one networkx object
* save 

In [ ]:
# https://geofa.geodanmark.dk/ows/fkg/fkg

In [ ]:
from owslib.wfs import WebFeatureService
import os
from qgis.core import QgsVectorLayer
from qgis import processing

In [ ]:
def fix_geometries(input_layer):
    """
    Fix invalid geometries in input layer and return temporary layer with valid geoms

    Arguments:
        input_layer (vector layer): layer with (potentially) invalid geoms

    Returns:
        fixed_layer: vector layer with valid geoms
    """

    fixed_layer = processing.run(
        "native:fixgeometries", {"INPUT": input_layer, "OUTPUT": "TEMPORARY_OUTPUT"}
    )["OUTPUT"]

    return fixed_layer


def clip_save_layer(input_layer, study_area_vlayer, filepath, layer_name):
    """
    Clip input layer with vector layer and save as geopackage

    Arguments:
        input_layer (vector layer): layer to be clipped
        study_area_vlayer (vector layer): vector layer defining clip extent
        filepath (str): filepath for saving clipped layer
        layer_name (str): name of layer for print statement

    Returns:
        None
    """
    clip_params = {
        "INPUT": input_layer,
        "OVERLAY": study_area_vlayer,
        "OUTPUT": filepath,
    }

    # clip to study area polygon
    processing.run("native:clip", clip_params)

    print(f"Saved layer {layer_name}")

    return None


def get_wfs_layers(
    study_area_vlayer, bounds, wfs_core, wfs_name, wfs_version, homepath, proj_crs
):
    """
    - creates a new subdir for WFS connection
    - downloads all available layers from the WFS connection
    - clips all layers to the extent of study area
    - saves all layers to new directory as geopackage

    Arguments:
        study_area_vlayer (vector layer): vector layer defining the study area/clip extent
        bounds (tuple): bounds for WFS request
        wfs_core (str): base url for WFS connection. E.g. f"https://rida-services.test.septima.dk/ows?MAP={wfs_name}&service=WFS"
        wfs_name (str): name of WFS used to create new directory for storing data (usually same as the name used in the base WFS url)
        wfs_version (str): version of WFS for WFS request
        homepath (str): homepath for QGIS project
        proj_crs (str): CRS in the format "EPSG:XXXX" used for WFS request

    Returns:
        None
    """

    # define bounds
    minx, miny, maxx, maxy = bounds

    # define WFS URL
    wfs_url_get = wfs_core + "&request=GetCapabilities"
    wfs = WebFeatureService(url=wfs_url_get, version=wfs_version)

    layers_to_import = list(wfs.contents)

    print("Importing layers:", layers_to_import, "from WFS: ", wfs_name)

    wfs_dir = homepath + f"/data/raw/wfs/"

    if not os.path.isdir(wfs_dir):
        os.mkdir(wfs_dir)

    wfs_layer_dir = homepath + f"/data/raw/wfs/{wfs_name}/"

    if not os.path.isdir(wfs_layer_dir):
        os.mkdir(wfs_layer_dir)

    for layer in layers_to_import:
        filepath = wfs_layer_dir + layer + ".gpkg"

        print("Getting data for layer:", layer)

        wfs_url = (
            wfs_core
            + f"&request=GetFeature&typeName={layer}&SRSName=EPSG:25832&BBOX={minx},{miny},{maxx},{maxy}"
        )

        Source = f"pagingEnabled='true' preferCoordinatesForWfsT11='false' restrictToRequestBBOX='1' srsname={proj_crs} typename={layer} url={wfs_url} version='auto'"

        # initialize vector layer of WFS features
        temp_layer = QgsVectorLayer(Source, layer, "WFS")

        fixed_layer = fix_geometries(temp_layer)

        clip_save_layer(fixed_layer, study_area_vlayer, filepath, layer)

In [1]:
import geopandas as gpd
import folium
import momepy
import matplotlib.pyplot as plt
import shapely
import numpy as np
from esda.shape import convex_hull_ratio

In [2]:
proj_crs = 'EPSG:25832'
length_threshold = 750 # in meters

In [ ]:
gpd.__version__

In [4]:
nodes = gpd.read_file("../data/cykelknudepunkter.geojson") #.to_crs(proj_crs)
edges = gpd.read_file("../data/cykelknudepunktsstraekninger.geojson") #.to_crs(proj_crs)

In [5]:
nodes_old = gpd.read_file("../data/network-technical/nodes.gpkg")
edges_old = gpd.read_file("../data/network-technical/edges.gpkg")

In [ ]:
#m=nodes[["geometry"]].explore(tiles = "cartodb positron", name = "nodes", color = "red", prefer_canvas=True)
m=edges_old[["geometry"]].explore(name = "edges_old", color = "blue")
edges[["geometry"]].explore(m=m,name="edges", color = "green")
folium.LayerControl().add_to(m)
m

***

In [ ]:
geom = edges.buffer(20).union_all()
ser = gpd.GeoSeries([geom], crs = edges.crs)
bou = ser.boundary
bou = bou.explode().reset_index(drop=True)
bou = gpd.GeoDataFrame({"geometry":bou}, crs = edges.crs)

In [ ]:
# drop too-short linestrings
bou_red = bou[bou.length>length_threshold].copy()

# polygonize
poly = shapely.polygonize(
    np.array(
        bou_red.geometry
    )
)

gdf = gpd.GeoDataFrame(
    {
        "geometry": poly.geoms
    },
    crs = edges.crs
)

gdf["i"] = gdf.index

In [ ]:
gdf = gdf[[i!=[] for i in gdf.interiors]].reset_index(drop=True)

In [ ]:
gdf

***

# skeletonize separately

In [ ]:
delin = gpd.GeoDataFrame(
    {
        "geometry":
        [
            LineString(r) for r in gdf.interiors] + [LineString(highway.exterior)
        ]
    },
    crs = roads.crs
)
delin

In [ ]:
def unzip_line(geom, coordnum = 30):
    longline = [c for c in geom.coords]
    linestrings = []
    current_linestring = []
    for c in longline:
        current_linestring.append(c)
        if len(current_linestring) > coordnum:
            linestrings.append(LineString(current_linestring))
            del current_linestring
            current_linestring = [c]
    if current_linestring:
        linestrings.append(LineString(current_linestring))
    return linestrings

In [ ]:
all_lines = []

for geom in delin.geometry:
    all_lines+= (
        unzip_line(geom, my_coordnum)
    )

In [ ]:
lines = gpd.GeoDataFrame({"geometry": all_lines}, crs = roads.crs)
lines["i"] = lines.index
lines.explore(tiles="cartodb positron", column="i")

In [ ]:
skel = voronoi_skeleton(
    lines = lines.geometry,
    poly = highway,
    max_segment_length = my_maxseglen
)

In [ ]:
gdf.interiors

In [ ]:
gdf.filter([16,18,21], axis = 0).plot()